# PLAYGROUND FOR DEVELOPING THIS APPS, DELETE OR IGNORE THIS NOTEBOOK/FILE.

In [5]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Any

import joblib
import os
import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
movies = pd.read_csv('../raw/tmdb_5000_movies.csv')
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [10]:
creds = pd.read_csv('../raw/tmdb_5000_credits.csv')
creds

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [11]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [12]:
creds.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [90]:
staging = pd.merge(movies[['genres', 'id', 'keywords', 'original_language', 'title', 'overview', 'production_companies', 'runtime', 'spoken_languages', 'tagline']], creds[['movie_id', 'cast']], left_on='id', right_on='movie_id').drop(columns=['movie_id', 'id'])
staging['runtime'] = staging['runtime'].astype(str)
staging.head()

,genres,keywords,original_language,title,overview,production_companies,runtime,spoken_languages,tagline,cast
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.","[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


In [91]:
jsonCols = ['genres', 'keywords', 'production_companies', 'spoken_languages', 'cast']
master = staging.copy(True)
for col in jsonCols:
    master[col] = master[col].map(lambda x : [g['name'] for g in json.loads(x)])
    master[col] = master[col].map(lambda x : ', '.join(x))

master


,genres,keywords,original_language,title,overview,production_companies,runtime,spoken_languages,tagline,cast
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","Ingenious Film Partners, Twentieth Century Fox...",162.0,"English, Español",Enter the World of Pandora.,"Sam Worthington, Zoe Saldana, Sigourney Weaver..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Walt Disney Pictures, Jerry Bruckheimer Films,...",169.0,English,"At the end of the world, the adventure begins.","Johnny Depp, Orlando Bloom, Keira Knightley, S..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",en,Spectre,A cryptic message from Bond’s past sends him o...,"Columbia Pictures, Danjaq, B24",148.0,"Français, English, Español, Italiano, Deutsch",A Plan No One Escapes,"Daniel Craig, Christoph Waltz, Léa Seydoux, Ra..."
3,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"Legendary Pictures, Warner Bros., DC Entertain...",165.0,English,The Legend Ends,"Christian Bale, Michael Caine, Gary Oldman, An..."
4,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",en,John Carter,"John Carter is a war-weary, former military ca...",Walt Disney Pictures,132.0,English,"Lost in our world, found in another.","Taylor Kitsch, Lynn Collins, Samantha Morton, ..."
...,...,...,...,...,...,...,...,...,...,...
4798,"Action, Crime, Thriller","united states–mexico barrier, legs, arms, pape...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,Columbia Pictures,81.0,Español,"He didn't come looking for trouble, but troubl...","Carlos Gallardo, Jaime de Hoyos, Peter Marquar..."
4799,"Comedy, Romance",,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,,85.0,,A newlywed couple's honeymoon is upended by th...,"Edward Burns, Kerry Bishé, Marsha Dietlein, Ca..."
4800,"Comedy, Drama, Romance, TV Movie","date, love at first sight, narration, investig...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","Front Street Pictures, Muse Entertainment Ente...",120.0,English,NaN,"Eric Mabius, Kristin Booth, Crystal Lowe, Geof..."
4801,,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,98.0,English,A New Yorker in Shanghai,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ..."


In [15]:
master.columns

Index(['genres', 'keywords', 'original_language', 'original_title', 'overview',
       'production_companies', 'runtime', 'spoken_languages', 'tagline',
       'cast'],
      dtype='object')

In [93]:
master['oneLiner'] = master.apply(lambda row : f'A movie with the title of {row["title"]}, the genres are {row["genres"]} with keywords of {row["keywords"]}, the casts are {row["cast"]}, spoken in {row["spoken_languages"]}, with {row["tagline"]} as the tagline.', axis=1)

In [117]:
master.head()

,genres,keywords,original_language,title,overview,production_companies,runtime,spoken_languages,tagline,cast,oneLiner
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","Ingenious Film Partners, Twentieth Century Fox...",162.0,"English, Español",Enter the World of Pandora.,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","A movie with the title of Avatar, the genres a..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Walt Disney Pictures, Jerry Bruckheimer Films,...",169.0,English,"At the end of the world, the adventure begins.","Johnny Depp, Orlando Bloom, Keira Knightley, S...",A movie with the title of Pirates of the Carib...
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",en,Spectre,A cryptic message from Bond’s past sends him o...,"Columbia Pictures, Danjaq, B24",148.0,"Français, English, Español, Italiano, Deutsch",A Plan No One Escapes,"Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","A movie with the title of Spectre, the genres ..."
3,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"Legendary Pictures, Warner Bros., DC Entertain...",165.0,English,The Legend Ends,"Christian Bale, Michael Caine, Gary Oldman, An...",A movie with the title of The Dark Knight Rise...
4,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",en,John Carter,"John Carter is a war-weary, former military ca...",Walt Disney Pictures,132.0,English,"Lost in our world, found in another.","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","A movie with the title of John Carter, the gen..."


In [83]:
master[master['original_title'] == '七人の侍']['oneLiner'].values

array(['A movie with the title of 七人の侍, the genres are Action, Drama with keywords of japan, samurai, peasant, looting, rice, fencing, village, moral ambiguity, battle, practice, 16th century, the casts are Toshirō Mifune, Takashi Shimura, Yoshio Inaba, Seiji Miyaguchi, Minoru Chiaki, Daisuke Katô, Isao Kimura, Keiko Tsushima, Yukiko Shimazaki, Kamatari Fujiwara, Yoshio Kosugi, Bokuzen Hidari, Yoshio Tsuchiya, Kokuten Kôdô, Eijirô Tôno, Jun Tatara, Atsushi Watanabe, Toranosuke Ogawa, Isao Yamagata, Kichijirô Ueda, Sôjin Kamiyama, Gen Shimizu, Keiji Sakakida, Shinpei Takagi, Shin Ôtomo, Toshio Takahara, Hiroshi Sugi, Hiroshi Hayashi, Sachio Sakai, Sôkichi Maki, Ichirô Chiba, Noriko Sengoku, Noriko Honma, Masanobu Ôkubo, Etsuo Saijo, Minoru Itô, Haruya Sakamoto, Gorô Sakurai, Hideo Shibuya, Kiyoshi Kamoda, Senkichi Ômura, Takashi Narita, Shôichi Hirose, Kôji Uno, Masaaki Tachibana, Kamayuki Tsubono, Taiji Naka, Chindanji Miyagawa, Shigemi Sunagawa, Akira Tani, Akio Kusama, Ryûtarô Amami,

In [18]:
class lemmaToken():
    def __init__(self) -> None:
        self.lemma = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('english'))
    def getPos(self, tags:str):
        if tags.startswith('J'):
            return wordnet.ADJ
        elif tags.startswith('V'):
            return wordnet.VERB
        elif tags.startswith('N'):
            return wordnet.NOUN
        elif tags.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    def __call__(self, doc) -> Any:
        tokens = word_tokenize(doc)
        wnt = nltk.pos_tag(tokens)
        return [self.lemma.lemmatize(word, pos=self.getPos(tag)) for word, tag in wnt if word.lower() not in self.stopwords]

In [94]:
title2idx = pd.Series(master.index, index=master['title'])

In [115]:
tfidfVec = TfidfVectorizer(stop_words='english')
vectored = tfidfVec.fit_transform(master['oneLiner'])

In [106]:
cobaIdx = vectored[title2idx['Spectre']]
cobaIdx

<1x30000 sparse matrix of type '<class 'numpy.float64'>'
	with 168 stored elements in Compressed Sparse Row format>

In [117]:
master['tfidf'] = master['oneLiner'].map(lambda x : vectored.transform([x]))
master

,genres,keywords,original_language,original_title,overview,production_companies,runtime,spoken_languages,tagline,cast,oneLiner,tfidf
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","Ingenious Film Partners, Twentieth Century Fox...",162.0,"English, Español",Enter the World of Pandora.,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","A movie with the title of Avatar, the overview...","(0, 58002)\t0.03418845408859126\n (0, 57739..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Walt Disney Pictures, Jerry Bruckheimer Films,...",169.0,English,"At the end of the world, the adventure begins.","Johnny Depp, Orlando Bloom, Keira Knightley, S...",A movie with the title of Pirates of the Carib...,"(0, 57597)\t0.07978672546456343\n (0, 56963..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",en,Spectre,A cryptic message from Bond’s past sends him o...,"Columbia Pictures, Danjaq, B24",148.0,"Français, English, Español, Italiano, Deutsch",A Plan No One Escapes,"Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","A movie with the title of Spectre, the overvie...","(0, 58295)\t0.027945049843243922\n (0, 5796..."
3,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"Legendary Pictures, Warner Bros., DC Entertain...",165.0,English,The Legend Ends,"Christian Bale, Michael Caine, Gary Oldman, An...",A movie with the title of The Dark Knight Rise...,"(0, 57989)\t0.037559445077635964\n (0, 5797..."
4,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",en,John Carter,"John Carter is a war-weary, former military ca...",Walt Disney Pictures,132.0,English,"Lost in our world, found in another.","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","A movie with the title of John Carter, the ove...","(0, 56963)\t0.06615353093334687\n (0, 56887..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,"Action, Crime, Thriller","united states–mexico barrier, legs, arms, pape...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,Columbia Pictures,81.0,Español,"He didn't come looking for trouble, but troubl...","Carlos Gallardo, Jaime de Hoyos, Peter Marquar...","A movie with the title of El Mariachi, the ove...","(0, 56945)\t0.05435426018006247\n (0, 55788..."
4799,"Comedy, Romance",,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,,85.0,,A newlywed couple's honeymoon is upended by th...,"Edward Burns, Kerry Bishé, Marsha Dietlein, Ca...","A movie with the title of Newlyweds, the overv...","(0, 54312)\t0.3464001187000969\n (0, 52569)..."
4800,"Comedy, Drama, Romance, TV Movie","date, love at first sight, narration, investig...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","Front Street Pictures, Muse Entertainment Ente...",120.0,English,NaN,"Eric Mabius, Kristin Booth, Crystal Lowe, Geof...","A movie with the title of Signed, Sealed, Deli...","(0, 58096)\t0.12791240678585308\n (0, 56963..."
4801,,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,98.0,English,A New Yorker in Shanghai,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...","A movie with the title of Shanghai Calling, th...","(0, 57920)\t0.1479719494019454\n (0, 57499)..."


In [108]:
query = master[master['original_title'] == 'Avatar']['oneLiner'].squeeze()
# query
x = tfidfVec.transform([query])
arrayScore = np.array(master[['original_title', 'tfidf']])

In [110]:
print(cosine_similarity(x, arrayScore[2][1]))

[[0.55044506]]


AttributeError: 'numpy.int64' object has no attribute 'index'

In [63]:
res = cosine_similarity(vectored[title2idx['Avatar']], vectored).flatten()
res = title2idx[(-res).argsort()]
res

'Avatar'

In [114]:
def getScore(title, matrix, idxIndices):
    query = matrix[idxIndices[title]]
    res = cosine_similarity(query, matrix).flatten()
    return idxIndices[(-res).argsort()][1:6].index
    # queryVec = tfidfVec.transform([query])
    # res = []
    # for i in range(len(matrix)):
    #     if matrix[i][0] == title:
    #         continue
    #     else:
    #         res.append([matrix[i][0], cosine_similarity(queryVec, matrix[i][1])[0][0]])
    # for key, val in idxIndices.
    # res = cosine_similarity(query, matrix).flatten()
    # res.sort(key=lambda x: x[1], reverse=True)

    # return [idxIndices[idx] for idx in res[:5]]

res = getScore('Avatar', vectored, title2idx)
res

Index(['Alien', 'Aliens', 'Alien³', 'Planet of the Apes', 'Space Cowboys'], dtype='object', name='title')

In [116]:
joblib.dump(vectored, 'tfidf2.pkl')

['tfidf2.pkl']

In [119]:
master.to_pickle('data2.pkl')

In [138]:
master['tfidf'].to_csv('coba.csv')

In [141]:
cobaUp = pd.read_csv('coba.csv')
cobaUp['tfidf'].dtypes

dtype('O')

In [143]:
cobaIm = pd.read_pickle('data.pkl')
cobaIm

,genres,keywords,original_language,original_title,overview,production_companies,runtime,spoken_languages,tagline,cast,oneLiner,tfidf
0,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","Ingenious Film Partners, Twentieth Century Fox...",162.0,"English, Español",Enter the World of Pandora.,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","A movie with the title of Avatar, the overview...","(0, 58002)\t0.03418845408859126\n (0, 57739..."
1,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Walt Disney Pictures, Jerry Bruckheimer Films,...",169.0,English,"At the end of the world, the adventure begins.","Johnny Depp, Orlando Bloom, Keira Knightley, S...",A movie with the title of Pirates of the Carib...,"(0, 57597)\t0.07978672546456343\n (0, 56963..."
2,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...",en,Spectre,A cryptic message from Bond’s past sends him o...,"Columbia Pictures, Danjaq, B24",148.0,"Français, English, Español, Italiano, Deutsch",A Plan No One Escapes,"Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","A movie with the title of Spectre, the overvie...","(0, 58295)\t0.027945049843243922\n (0, 5796..."
3,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"Legendary Pictures, Warner Bros., DC Entertain...",165.0,English,The Legend Ends,"Christian Bale, Michael Caine, Gary Oldman, An...",A movie with the title of The Dark Knight Rise...,"(0, 57989)\t0.037559445077635964\n (0, 5797..."
4,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...",en,John Carter,"John Carter is a war-weary, former military ca...",Walt Disney Pictures,132.0,English,"Lost in our world, found in another.","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","A movie with the title of John Carter, the ove...","(0, 56963)\t0.06615353093334687\n (0, 56887..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4798,"Action, Crime, Thriller","united states–mexico barrier, legs, arms, pape...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,Columbia Pictures,81.0,Español,"He didn't come looking for trouble, but troubl...","Carlos Gallardo, Jaime de Hoyos, Peter Marquar...","A movie with the title of El Mariachi, the ove...","(0, 56945)\t0.05435426018006247\n (0, 55788..."
4799,"Comedy, Romance",,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,,85.0,,A newlywed couple's honeymoon is upended by th...,"Edward Burns, Kerry Bishé, Marsha Dietlein, Ca...","A movie with the title of Newlyweds, the overv...","(0, 54312)\t0.3464001187000969\n (0, 52569)..."
4800,"Comedy, Drama, Romance, TV Movie","date, love at first sight, narration, investig...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","Front Street Pictures, Muse Entertainment Ente...",120.0,English,NaN,"Eric Mabius, Kristin Booth, Crystal Lowe, Geof...","A movie with the title of Signed, Sealed, Deli...","(0, 58096)\t0.12791240678585308\n (0, 56963..."
4801,,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,98.0,English,A New Yorker in Shanghai,"Daniel Henney, Eliza Coupe, Bill Paxton, Alan ...","A movie with the title of Shanghai Calling, th...","(0, 57920)\t0.1479719494019454\n (0, 57499)..."
